In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [2]:
import json

In [3]:
with open("bidder-data/data152.json") as f:
    ex = json.load(f)


In [4]:
ex[:3]

[{'rules': ['DefaultPass',
   'OneLevelSuitOpening',
   'OneLevelTakeoutDouble',
   'DefaultPass',
   'SuitResponseToTakeoutDouble',
   'UnforcedRebidOriginalSuitByOpener',
   'LawOfTotalTricks',
   'DefaultPass',
   'LawOfTotalTricks',
   'DefaultPass',
   'DefaultPass',
   'DefaultPass'],
  'board': '4-ba4b7058ecc802db04967b7b5d',
  'calls': ['P', '1D', 'X', 'P', '1S', '2D', '2S', 'P', '4S', 'P', 'P', 'P']},
 {'rules': ['DefaultPass',
   'OneLevelSuitOpening',
   'DefaultPass',
   'OneLevelNewSuitResponse',
   'DefaultPass',
   'RebidOneNotrumpByOpener',
   'DefaultPass',
   'NaturalNotrump',
   'DefaultPass',
   'DefaultPass',
   'DefaultPass'],
  'board': '12-5ca2c074df721394da2eb47e29',
  'calls': ['P', '1C', 'P', '1S', 'P', '1N', 'P', '2N', 'P', 'P', 'P']},
 {'rules': ['OneLevelSuitOpening',
   'DefaultPass',
   'OneLevelNewSuitResponse',
   'DefaultPass',
   'NewSuitByOpener',
   'DefaultPass',
   'ThreeLevelSuitRebidByResponder',
   'DefaultPass',
   'NaturalNotrump',
   'Defau

In [6]:
import game_engine